## LightGBM算法的核心思想
LigthGBM算法是Boosting算法的新成员，由微软公司开发。它和XGBoost算法一样是对GBDT算法的高效实现，在原理上与GBDT算法和XGBoost算法类似，都采用损失函数的负梯度作为当前决策树的残差近似值，去拟合新的决策树。

**与传统的机器学习算法相比，LightGBM算法具有这些优势：训练效率更高；低内存使用；准确率更高；支持并行化学习；可以处理大规模数据。**

## LightGBM算法的数学原理概述
1．基于leaf-wise的决策树生长策略大部分决策树算法使用的生长策略是level-wise生长策略，即同一层的叶子节点每次都一起分裂，如下图所示。但实际上一些叶子节点的分裂增益较低，这样分裂会增加不小的开销。

### 1．基于leaf-wise的决策树生长策略
大部分决策树算法使用的生长策略是level-wise生长策略，即同一层的叶子节点每次都一起分裂，如下图所示。但实际上一些叶子节点的分裂增益较低，这样分裂会增加不小的开销。
![基于leaf-wise的决策树生长策略1](image/基于leaf-wise的决策树生长策略1.png)
LightGBM算法使用的则是leaf-wise生长策略，每次在当前叶子节点中找出分裂增益最大的叶子节点进行分裂，而不是所有节点都进行分裂，如下图所示，这样可以提高精度。但是，leaf-wise策略在样本量较小时容易造成过拟合，LightGBM算法可以通过参数max_depth限制树的深度来防止过拟合。
![基于leaf-wise的决策树生长策略2](image/基于leaf-wise的决策树生长策略2.png)

### 2．直方图算法
直方图分为频数直方图和频率直方图，横坐标为相关数据，纵坐标为该数据出现的频数或频率。如用hist()函数绘制了下图所示的频数直方图。
![直方图算法](image/直方图算法.png)
直方图算法又称为histogram算法，简单来说，就是先对特征值进行装箱处理，将连续的浮点特征值离散化成k个整数，形成一个个箱体（bins），同时构造一个宽度为k的直方图，在遍历数据时，以离散化后的值作为索引在直方图中累积统计量（因此这里是频数直方图）。遍历一次数据后，直方图累积了需要的统计量，再根据直方图的离散值遍历寻找最优分割点。

对于连续特征来说，装箱处理就是特征工程中的离散化（可以参考11.4节的数据分箱），例如，[0，10）区间的值都赋值为0，[10，20）区间的值都赋值为1等，这样就可以把众多的数值划分到有限的分箱中。LightGBM算法中默认的分箱数（bins）为256。

举例来说，现在有10000个客户，也就有10000个身高数据，将身高分箱为256份后（例如，身高180～180.2cm的所有客户都分箱为数字200），就变为256个数字，这时再统计每个数值对应的频数（例如，身高180～180.2cm的客户为100人，那么数字200对应的频数就是100）。这样在节点分裂时，就不需要按照预排序算法对每个特征都计算10000遍（样本总数），而只需要计算256遍（分箱数），大大加快了训练速度。

对于分类特征来说，则是将每一种取值放入一个分箱（bin），且当取值的个数大于最大分箱数时，会忽略那些很少出现的分类值。例如，10000个客户的国籍数据，便可以按国家名称进行分箱，如果超过最大分箱数（如256），那么很少出现的国家就会被忽略。

### 3．并行学习
LightGBM算法支持特征并行和数据并行两种学习方式。传统的特征并行的主要思想是在并行化决策树中寻找最佳切分点，在数据量大时难以加速，同时需要对切分结果进行通信整合。而LightGBM算法在本地保存全部数据，这样就没有了机器间通信所需的开销。此外，传统的数据并行是构建本地直方图，然后进行整合，在全局直方图中寻找最佳切分点。LightGBM算法则使用分散规约（reduce scatter），将直方图合并的任务分给不同的机器，降低通信和计算的开销，并利用直方图做加速训练，进一步减少开销。

除了上述原理，LightGBM算法还包含一些重要的算法思想，如单边梯度采样GOSS算法（Gradient-based One-Side Sampling）和互斥特征绑定EFB算法（Exclusive Feature Bundling）。在GOSS算法中，梯度更大的样本点在计算信息增益时会发挥更重要的作用，当对样本进行下采样时保留这些梯度较大的样本点，并随机去掉梯度小的样本点。EFB算法则将互斥特征绑在一起以减少特征维度。

## LightGBM算法的简单代码实现
LightGBM算法既能做分类分析，又能做回归分析，对应的模型分别为LightGBM分类模型（LGBMClassifier）和LightGBM回归模型（LGBMRegressor）。

> 安装LightGBM
（https://www.cnblogs.com/ironan-liu/p/11957866.html）

In [1]:
from lightgbm import LGBMClassifier

In [2]:
LGBMClassifier?

### 一、LightGBM分类模型的简单演示代码

In [ ]:
from lightgbm import LGBMClassifier
x = [[1,2], [3,4], [5,6], [7,8], [9,10]]
y = [0, 0, 0, 1, 1] 
model = LGBMClassifier()
model.fit(x, y)
print(model.predict([[5,5]]))

###  二、LightGBM回归模型的简单演示代码

In [ ]:
from lightgbm import LGBMRegressor
X = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
y = [1, 2, 3, 4, 5]
model = LGBMRegressor()
model.fit(X, y)
print(model.predict([[5, 5]]))